## LSTM Captioning

This is a very basic model: 

*  Take the featurized images (2048d), and tokenised captions
*  Add a (trainable) features -> 50d dense layer
*  Use a 50d GloVe embedding (for the LSTM inputs, non-trainable)
   *  #stop-words ~ 150 (say)
*  50d of hidden units for the LSTM
*  But have a 'pluggable' output transform :
   *   Concat : (256 one-hot - including '0'=mask, '1'={UNK}, '2'={START}, '3'={STOP}, '4'={UseOther})
   *   (a) UseOther + (8192-250 of more one-hot)
   *   (b) UseOther + (50d of same GloVe embedding, for nearest-neighbour)
   *   (c) UseOther + (log2(8192)==13 bits + error correction of index of word)
*  Want to monitor some kind of score over time for test cases   

In [ ]:
import os

import numpy as np

import pickle

In [ ]:
# Load in the captions/corpus/embedding
with open('./data/cache/CAPTIONS_data_Flickr30k_2017-06-07_23-15.pkl', 'rb') as f:
    text_data=pickle.load(f, encoding='iso-8859-1')

"""
text_data ~ dict(
    img_arr = img_arr_save,
    img_to_captions = img_to_valid_captions,
    
    action_words = action_words, 
    stop_words = stop_words_sorted,
    
    embedding = embedding,
    embedding_word_arr = embedding_word_arr,
    
    train_test = np.random.random( (len(img_arr_save),) ),
)"""

In [ ]:
# Load in the features
with open('./data/cache/FEATURES_data_Flickr30k_flickr30k-images_2017-06-06_18-07.pkl', 'rb') as f:
    image_data=pickle.load(f, encoding='iso-8859-1')

"""
image_data ~ dict(
    features = features,
    img_arr = img_arr,
)
"""